In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import sklearn
import neptune.new as neptune

In [ ]:

import os,sys,pickle,time,random,glob
sys.path.append(os.getcwd())
import numpy as np
import pandas as pd

from typing import List
import copy
import os.path as osp
import torch
import torch.utils.data
#from torch_sparse import SparseTensor, cat
from torch_geometric.data import Data, Dataset
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
import torch.nn as nn

In [ ]:
Data(x=torch.from_numpy(X_train_split_element.astype("float32")), edge_index=torch.from_numpy(edge_index_train).type(torch.LongTensor), y = torch.from_numpy(obs_train_split_element).type(torch.LongTensor)))

In [ ]:
DataLoader(data_list_train, batch_size=2)

In [ ]:

parameters = {
    "learning_rate": 0.0001,
    "weight_decay": 0.05,
    "epoch_num": 100,
    "batch_size": 256,
    "number_attention_heads": 8,
    "dropout": 0.4,
    "number_hidden_units": 8,
    "number_node_features": 9,
    "number_classes": 2
}

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, parameters):
        super(GAT, self).__init__()
        self.gat1 = GATConv(parameters["number_node_features"], out_channels=parameters["number_hidden_units"],
                            heads=parameters["number_attention_heads"], concat=True, negative_slope=0.001,
                            dropout=parameters["dropout"], bias=True)
        self.gat2 = GATConv(parameters["number_hidden_units"] * parameters["number_attention_heads"], parameters["number_classes"], #n_nodes is missing
                            heads=parameters["number_attention_heads"], concat=False, negative_slope=0.001,
                            dropout=parameters["dropout"], bias=True)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        #x = self.gat1(x.float(), edge_index.float())
        #x = self.gat1(x.double(), edge_index.double())
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
import neptune.new as neptune

run = neptune.init(
    project="iGEM2021/GNN",
    tags="GNN01",
    api_token="",
    source_files=["*.py"],
)

#neptune.create_experiment()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = GAT(parameters).to(device)

criterion = nn.NLLLoss()
optimizer = torch.optim.Adagrad(model.parameters(),
                                lr=parameters["learning_rate"],
                                weight_decay=parameters["weight_decay"])

In [ ]:
step = 0
run["parameters"] = parameters
model.train()   

epoch_loss = []
epoch_acc = []
epoch_roc = []
outputs_train_list = []
epoch_test_loss = []
epoch_test_acc = []
epoch_test_roc = []
outputs_test_list = []

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
        
    for graph_train_batch in dataloader_train:
        graph_train_batch = graph_train_batch.to(device)

        optimizer.zero_grad()
        
        outputs = model(graph_train_batch) ########################
        
        max_val, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, graph_train_batch.y)
        
        acc = (torch.sum(preds == graph_train_batch.y)) / len(graph_train_batch.x)
        
        roc_score = roc_auc_score(graph_train_batch.y.clone().detach().cpu(), max_val.clone().detach().cpu()) #outputs.T[0].clone().detach().cpu()
        
        run["logs/training/batch/loss"].log(loss)
        print("Train loss")
        print(loss.item())
        #neptune.log_metric('loss', loss)
        run["logs/training/batch/acc"].log(acc)
        #neptune.log_metric('acc', acc)
        print("Train accuracy")
        print(acc.item())
        run["logs/training/batch/roc"].log(roc_score)
        #neptune.log_metric('roc', roc_score)
        print("Train roc")
        print(roc_score)
        
        epoch_loss.append(loss.item())
        epoch_acc.append(acc.item())
        epoch_roc.append(roc_score)
        outputs_train_list.append(outputs)
        
        loss.backward()
        optimizer.step()

        #writer.add_scalar('Training loss step', scores, global_step = step)

        step +=1
        
    #EVALUATION   
    #if step % 8 == 0:
    print("Evaluation")
    with torch.no_grad():           
        model.eval()
        scores = 0
        dimension = 0
        y_true = []
        y_pred = []

        for graph_test_batch in dataloader_test:

            graph_test_batch = graph_test_batch.to(device)

            outputs = model(graph_test_batch)

            max_val, preds = torch.max(outputs, 1)

            loss = criterion(outputs, graph_test_batch.y)

            acc = (torch.sum(preds == graph_test_batch.y)) / len(graph_test_batch.x)

            roc_score = roc_auc_score(graph_test_batch.y.clone().detach().cpu(), max_val.clone().detach().cpu())

            run["logs/test/batch/loss"].log(loss)
            #neptune.log_metric('test_loss', loss)
            run["logs/test/batch/acc"].log(acc)
            #neptune.log_metric('test_acc', acc)
            run["logs/test/batch/roc"].log(roc_score)
            #neptune.log_metric('roc', roc_score)

            epoch_test_loss.append(loss.item())
            epoch_test_acc.append(acc.item())
            epoch_test_roc.append(roc_score)
            
            outputs_test_list.append(outputs)

            #writer.add_scalar('Test loss step', scores/(dimension*output.size()[1]*output.size()[2]), global_step = step)

        model.train()   

        torch.save(model.state_dict(), f"./{epoch}-{step}-{set}.pt")
        
print("Max train loss epoch")
print(max(epoch_loss))
print("Max train acc epoch")
print(max(epoch_acc))
print("Max train roc epoch")
print(max(epoch_roc))
print("Max test loss epoch")
print(max(epoch_test_loss))
print("Max test acc epoch")
print(max(epoch_test_acc))
print("Max test roc epoch")
print(max(epoch_test_roc))